In [2]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
from account_credentials import LOGIN, PASSWORD, SERVER
from datetime import datetime, timedelta

In [3]:
is_initialized = mt5.initialize()
print('initialize: ', is_initialized)

is_logged_in = mt5.login(LOGIN, PASSWORD, SERVER)
print('logged in: ', is_logged_in)
print('\n')
account_info = mt5.account_info()
print(datetime.now(),
    '| Login: ', account_info.login,
    '| Balance: ', account_info.balance,
    '| Equity: ' , account_info.equity)

initialize:  True
logged in:  True


2023-04-20 07:09:09.422551 | Login:  114123121 | Balance:  532.12 | Equity:  529.7


In [4]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta

# connect to MetaTrader 5
mt5.initialize()

# set the time range for the history
start_time = datetime.now() - timedelta(days=100)
end_time = datetime.now()

# retrieve the history
history = mt5.history_deals_get(start_time, end_time)

# Create an empty dataframe
df = pd.DataFrame(columns=['Time','Deal ID', 'Order Ticket', 'Type', 'Volume', 'Price', 'Profit', 'Comment', 'Magic'])

# Iterate through the trade history and create a list of dataframes
dfs = []
for deal in history:
    row = {'Time': pd.Timestamp(deal.time, unit='s'),
           'Deal ID': deal.order,
           'Order Ticket': deal.ticket,
           'Type': 'Buy' if deal.type == 0 else 'Sell',
           'Volume': deal.volume,
           'Price': deal.price,
           'Profit': deal.profit,
           'Comment': deal.comment,
           'Magic': deal.magic}
    dfs.append(pd.DataFrame(row, index=[0]))

# Concatenate all dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Print the resulting dataframe
print(df)



                   Time    Deal ID  Order Ticket  Type  Volume     Price   
0   2023-03-23 01:46:47          0     143696145  Sell    0.00     0.000  \
1   2023-03-28 02:09:05  266948417     145348938   Buy    0.01  1960.969   
2   2023-03-28 02:29:00  266953667     145353163   Buy    0.01  1960.427   
3   2023-03-28 03:03:56  266962782     145360604   Buy    0.02  1961.108   
4   2023-03-28 04:18:59  266982644     145377761  Sell    0.02  1958.718   
..                  ...        ...           ...   ...     ...       ...   
251 2023-04-19 17:00:02  278102573     152930023   Buy    0.01  1996.381   
252 2023-04-19 18:00:00  278118473     152940995  Sell    0.01  1994.012   
253 2023-04-19 20:00:00  278139142     152952691  Sell    0.01  1994.288   
254 2023-04-19 23:42:32  278173045     152971349  Sell    0.01  1993.256   
255 2023-04-19 23:42:50  278173158     152971389  Sell    0.01  1993.156   

     Profit                       Comment   Magic  
0    500.00  D-trial-USD-INT-98b286

In [5]:
df_adjust = df[(df['Profit'] != 0) & (abs(df['Profit']) > 3) & (abs(df['Profit']) != 500)]
df_adjust.reset_index(inplace=True)
df_adjust.to_csv('history_price.csv')
df_adjust

,index,Time,Deal ID,Order Ticket,Type,Volume,Price,Profit,Comment,Magic
0,4,2023-03-28 04:18:59,266982644,145377761,Sell,0.02,1958.718,-4.78,,0
1,56,2023-03-29 15:08:09,267851674,146078214,Sell,0.01,1966.148,-3.12,[sl 1966.148],123999
2,58,2023-03-29 17:41:33,267915037,146134790,Sell,0.01,1968.253,3.12,[tp 1968.253],123999
3,60,2023-03-29 19:01:04,267933799,146151811,Sell,0.01,1966.052,-3.13,[sl 1966.052],123999
4,76,2023-03-30 16:00:46,268440990,146546900,Sell,0.01,1975.363,3.12,[tp 1975.363],123999
...,...,...,...,...,...,...,...,...,...,...
85,245,2023-04-19 14:06:25,278008088,152856148,Sell,0.01,1995.217,4.36,[tp 1993.985],123993
86,246,2023-04-19 14:06:25,278008126,152856205,Sell,0.01,1995.232,4.28,[tp 1994.076],123999
87,248,2023-04-19 16:32:19,278091008,152921245,Buy,0.01,1993.940,-3.12,[sl 1993.940],123992
88,254,2023-04-19 23:42:32,278173045,152971349,Sell,0.01,1993.256,-3.12,[sl 1993.256],123993


In [56]:
#type SEll >>> คือ buy ที่ขายออก
#type Buy >>> คือ sell ที่ซื้อออก
def get_summary(df_adjust, Magic, TPSL_base, Type_see):
    if Type_see == 'Buy':
        Type_cal = 'Sell'
    if Type_see == 'Sell':
        Type_cal = 'Buy'
    Order_type = Type_see
    Samples = len(df_adjust[(df_adjust["Magic"] == Magic) & (df_adjust["Type"] == Type_cal )])
    buy = df_adjust[(df_adjust["Magic"] == Magic) & (df_adjust["Type"] == Type_cal)]
    buy_profit = buy["Profit"]
    Profit = round(sum(buy_profit), 2)
    Accuracy = (buy["Profit"] > 0).sum() / len(buy["Profit"])
    standard = ((buy["Profit"] > 0).sum() * TPSL_base) - ((buy["Profit"] < 0).sum() * TPSL_base)
    error = round(Profit - standard, 2)
    return Samples, Order_type, Profit, Accuracy, error



```
print("------MODEL--------")
# TP and SL should be 3.13
TPSL_base = 3.13

print("All Samples",len(df_adjust[df_adjust["Magic"] == 123999]))
m1_df = df_adjust[df_adjust["Magic"] == 123999]
Profit = sum(m1_df["Profit"])
print(f"Profit ${Profit:.2f}")
Accuracy = (m1_df["Profit"] > 0).sum() / len(m1_df)
print(f"Accuracy {Accuracy*100:.2f}%")


Lost_standard = ((m1_df["Profit"] < 0).sum() * TPSL_base) #Positive Value
Lost_actual = (sum(m1_df[m1_df["Profit"] < 0]["Profit"])) # Negative Value
Lost_error = Lost_standard + Lost_actual # if result = positive then Lost_standard > Lost_actual
print(f"Lost Error ${Lost_error:.2f}")

Win_standard = ((m1_df["Profit"] > 0).sum() * TPSL_base) #Positive Value
Win_actual = (sum(m1_df[m1_df["Profit"] > 0]["Profit"])) # Positive Value
Win_error = Win_actual - Win_standard # if result = positive then the actual win is higher than standard win
print(f"Win Error ${Win_error:.2f}")
```

In [91]:
summary_df = pd.DataFrame(columns=['Model', 'Samples', 'Order_type', 'Profit', 'Accuracy', 'error'])
#print("---------V1")
Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123999, 3.13, "Buy")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = 'v1'
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)

#print("--------V2")
Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123992, 3.13, "Buy")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = "v2_buy"
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)

Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123992, 3.13, "Sell")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = "v2_sell"
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)

#print("V2b")
Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123993, 3.13, "Buy")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit_buy: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = 'v2b_buy'
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)

Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123993, 3.13, "Sell")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit_buy: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = 'v2b_sell'
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)

#print("V2c")
Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123994, 3.13, "Buy")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit_buy: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = 'v2c_buy'
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)


Samples, Order_type, Profit, Accuracy, error = get_summary(df_adjust, 123994, 3.13, "Sell")
# print(f"Samples: {Samples}")
# print(f"Order_type: {Order_type}")
# print(f"Profit_buy: ${Profit:.2f}")
# print(f"Accuracy {Accuracy*100:.2f}%")
# print(f"Error buy ${error:.2f}")
Model = 'v2c_sell'
new_row = pd.DataFrame({'Model': Model, 'Samples': Samples, 'Order_type': Order_type, 'Profit': Profit, 'Accuracy': Accuracy, 'error': error}, index=[0])
summary_df = pd.concat([summary_df, new_row], axis=0,ignore_index=True)
summary_df


50.74999999999999


,Model,Samples,Order_type,Profit,Accuracy,error
0,v1,38,Buy,20.39,0.578947,1.61
1,v2_buy,14,Buy,20.41,0.714286,1.63
2,v2_sell,25,Sell,-4.05,0.520000,-7.18
3,v2b_buy,3,Buy,4.65,0.666667,1.52
4,v2b_sell,5,Sell,3.12,0.600000,-0.01
5,v2c_buy,1,Buy,3.12,1.000000,-0.01
6,v2c_sell,3,Sell,3.11,0.666667,-0.02


In [98]:
total_profit = summary_df['Profit'].sum()
print(f"Total_profit ${total_profit:.2f}")
total_win_rate = summary_df['Accuracy'].mean()
print(f"Average accuracy {total_win_rate*100:.2f}%")

Total_profit $50.75
Average accuracy 67.81%
